### **Sobre os dados**

Os dados a respeito da pandemia de COVID-19 foram compilados pelo centro de ciência de sistemas e engenharia da universidade americana **[John Hopkins](https://www.jhu.edu/)**. O contéudo foi atualizados com uma granularidade temporal de dias e geográfica de regiões de países (estados, condados, etc.). O website do projeto pode ser acessado em [https://systems.jhu.edu/research/public-health/ncov/](https://systems.jhu.edu/research/public-health/ncov/).

Abaixo estão descritos os dados derivados do seu processamento:

* **date:** data de referência;
* **state:** estado;
* **country:** país;
* **population:** população estimada;
* **confirmed:** número acumulado de infectados;
* **confirmed_1d:** número diário de infectados;
* **confirmed_moving_avg_7d:** média móvel de 7 dias do número diário de infectados;
* **confirmed_moving_avg_7d_rate_14d:** média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
* **deaths:** número acumulado de mortos;
* **deaths_1d:** número diário de mortos;
* **deaths_moving_avg_7d:** média móvel de 7 dias do número diário de mortos;
* **deaths_moving_avg_7d:** média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
* **month:** mês de referência;
* **year:** ano de referência.

Já os dados sobre vacinação da COVID-19 foram compilados pelo projeto Nosso Mundo em Dados (*Our World in Data ou OWID*) da universidade britânica de **[Oxford](https://www.ox.ac.uk)**. Os dados foram atualizados diariamente deste janeiro de 2020 também e com uma granularidade temporal de dias e geográfica de países. O website do projeto pode ser acessado neste [https://ourworldindata.org](https://ourworldindata.org).

Abaixo estão descritos os dados derivados do seu processamento:

* **date:** data de referência;
* **state:** estado;
* **country:** país;
* **population:** população estimada;
* **confirmed:** número acumulado de infectados;
* **confirmed_1d:** número diário de infectados;
* **confirmed_moving_avg_7d:** média móvel de 7 dias do número diário de infectados;
* **confirmed_moving_avg_7d_rate_14d:** média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
* **deaths:** número acumulado de mortos;
* **deaths_1d:** número diário de mortos;
* **deaths_moving_avg_7d:** média móvel de 7 dias do número diário de mortos;
* **deaths_moving_avg_7d:** média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
* **month:** mês de referência;
* **year:** ano de referência.

**Fontes**

  - Casos pela universidade John Hopkins ([link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports));
  - Vacinação pela universidade de Oxford ([link](https://covid.ourworldindata.org/data/owid-covid-data.csv)).

Pacotes e bibliotecas

Nesta sessão vamos utilizar os seguintes pacotes Python para processar os dados bruto em um formato adequado para exploração interativa de dados no looker studio.

In [ ]:
import math
import numpy as np
import pandas as pd

from typing import Iterator
from datetime import datetime, timedelta

## **ETL dos casos de covid**

**1\. Extração**

In [ ]:
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/01-12-2021.csv', sep=',')

In [ ]:
cases.head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-13 05:22:15,33.93911,67.709953,53584,2301,44608,6675,Afghanistan,137.647787,4.294192
1,NaN,NaN,NaN,Albania,2021-01-13 05:22:15,41.15330,20.168300,64627,1252,38421,24954,Albania,2245.708527,1.937271
2,NaN,NaN,NaN,Algeria,2021-01-13 05:22:15,28.03390,1.659600,102641,2816,69608,30217,Algeria,234.067409,2.743543


In [ ]:
cases.fillna(0, inplace=True)

Os dados estão compilados em arquivos por dia, ou seja, cada dia corresponde a um arquivo.
Portanto precisaremos definir um intervalo de tempo, extraí-los e compilar os arquivos.
Para isso criamos uma lista com calendário gerado a partir do intervalo do nosso interesse:

In [ ]:
def calendar(dt_start: str, dt_end: str):

    dt_start = datetime.strptime(dt_start, "%m-%d-%Y")
    dt_end = datetime.strptime(dt_end, "%m-%d-%Y")

    delta = dt_end - dt_start
    list_dates = []

    for i in range(delta.days + 1):
        day = dt_start + timedelta(days=i)
        dt = datetime.strftime(day, format="%m-%d-%Y")
        list_dates.append(dt)

    return list_dates

In [ ]:
test_start_date = '01-01-2021'
test_end_date = '01-31-2021'

print(calendar(dt_start=test_start_date, dt_end=test_end_date))

['01-01-2021', '01-02-2021', '01-03-2021', '01-04-2021', '01-05-2021', '01-06-2021', '01-07-2021', '01-08-2021', '01-09-2021', '01-10-2021', '01-11-2021', '01-12-2021', '01-13-2021', '01-14-2021', '01-15-2021', '01-16-2021', '01-17-2021', '01-18-2021', '01-19-2021', '01-20-2021', '01-21-2021', '01-22-2021', '01-23-2021', '01-24-2021', '01-25-2021', '01-26-2021', '01-27-2021', '01-28-2021', '01-29-2021', '01-30-2021', '01-31-2021']


Nesta analise decidimos selecionar apenas os **dados referentes ao Brasil** e, além disso, **vamos selecionar apenas as colunas pertinentes**.

Agora sim vamos a extração. Ela deve demorar uns minutos pelo tamanho da base.

In [ ]:
start_date = '01-01-2021'
end_date = '12-31-2021'

In [ ]:
df_cases = None

for date in calendar(dt_start=start_date, dt_end=end_date):

    data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date}.csv'

    cases = pd.read_csv(data_source_url, sep=',')
    cases = cases.drop(['FIPS', 'Admin2', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)

    df_cases = pd.concat([df_cases, cases])

In [ ]:
df_cases = df_cases.query('Country_Region == "Brazil"').reset_index(drop=True)
df_cases.head(2)

,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Incident_Rate
0,Acre,Brazil,2021-01-02 05:22:33,41689,796,4726.992352
1,Alagoas,Brazil,2021-01-02 05:22:33,105091,2496,3148.928928


* Agora que os dados foram coletados é importante avaliarmos o conteúdo deste dataframe como formatos, se existem informações nulas, etc.

In [ ]:
df_cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9855 entries, 0 to 9854
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province_State  9855 non-null   object 
 1   Country_Region  9855 non-null   object 
 2   Last_Update     9855 non-null   object 
 3   Confirmed       9855 non-null   int64  
 4   Deaths          9855 non-null   int64  
 5   Incident_Rate   9855 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 462.1+ KB


In [ ]:
print(df_cases.isna().sum())
print((df_cases == '').sum())

Province_State    0
Country_Region    0
Last_Update       0
Confirmed         0
Deaths            0
Incident_Rate     0
dtype: int64
Province_State    0
Country_Region    0
Last_Update       0
Confirmed         0
Deaths            0
Incident_Rate     0
dtype: int64


### **Wrangling**

* O foco é garantir uma boa granularidade e qualidade da base de dados.

In [ ]:
df_cases = df_cases.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country',
    'Last_Update': 'date',
    'Confirmed': 'confirmed',
    'Deaths': 'deaths',
    'Incident_Rate': 'incident_rate'
  }
)

* Ajustamos o nome dos estados.

In [ ]:
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

df_cases['state'] = df_cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

* Vamos então criar novas colunas para enriquecer nossa a base de dados.

 - Chaves temporais

In [ ]:
df_cases['year-month'] = df_cases['date'].apply(lambda date: date[0:7])
df_cases['year']  = df_cases['date'].apply(lambda date: date[0:4])

- População estimada do estado

In [ ]:
df_cases['population'] = round(100000 * (df_cases['confirmed'] / df_cases['incident_rate']))
df_cases = df_cases.drop('incident_rate', axis=1)

- Média móvel (7 dias) de casos e mortes
- Estabilidade (14 dias) de casos e mortes

In [ ]:
def get_trend(rate: float) -> str:

    if np.isnan(rate):
        return np.NaN
    elif rate < 0.75:
        status = 'downward'
    elif rate > 1.15:
        status = 'upward'
    else:
        status = 'stable'

    return status

In [ ]:
df_cases_state = None
list_states = list(df_cases['state'].drop_duplicates())

for state in list_states:
        custom_query = f'state == "{state}"'
        cases_per_state = df_cases.query(custom_query).reset_index(drop=True)
        cases_per_state = cases_per_state.sort_values(by=['date'])

        cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
        cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
        cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
        cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

        cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
        cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
        cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
        cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

        df_cases_state = pd.concat([df_cases_state,cases_per_state])

- Garantir o tipo do dado é fundamental para consistência da base de dados. Vamos fazer o *type casting* das colunas.

In [ ]:
df_cases_state['population'] = df_cases_state['population'].astype('Int64')
df_cases_state['confirmed_1d'] = df_cases_state['confirmed_1d'].astype('Int64')
df_cases_state['confirmed_moving_avg_7d'] = df_cases_state['confirmed_moving_avg_7d'].astype('Int64')
df_cases_state['deaths_1d'] = df_cases_state['deaths_1d'].astype('Int64')
df_cases_state['deaths_moving_avg_7d'] = df_cases_state['deaths_moving_avg_7d'].astype('Int64')

- Por fim, vamos reorganizar as colunas e conferir o resultado final.

In [ ]:
df_cases = df_cases_state[['date'
                           ,'country'
                           ,'state'
                           ,'population'
                           ,'confirmed'
                           ,'confirmed_1d'
                           ,'confirmed_moving_avg_7d'
                           ,'confirmed_moving_avg_7d_rate_14d'
                           ,'confirmed_trend'
                           , 'deaths'
                           , 'deaths_1d'
                           , 'deaths_moving_avg_7d'
                           , 'deaths_moving_avg_7d_rate_14d'
                           , 'deaths_trend'
                           , 'year-month'
                           , 'year']]

df_cases.tail(5)

,date,country,state,population,confirmed,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend,year-month,year
360,2021-12-28 04:21:47,Brazil,Tocantins,1572866,234113,0,0,0.000000,downward,3927,0,0,0.0,downward,2021-12,2021
361,2021-12-29 03:21:43,Brazil,Tocantins,1572866,234964,851,122,2.837209,upward,3933,6,1,1.0,stable,2021-12,2021
362,2021-12-30 04:21:25,Brazil,Tocantins,1572866,235340,376,176,inf,upward,3936,3,2,inf,upward,2021-12,2021
363,2021-12-31 04:21:50,Brazil,Tocantins,1572866,235558,218,207,inf,upward,3939,3,2,inf,upward,2021-12,2021
364,2022-01-01 04:22:14,Brazil,Tocantins,1572866,235558,0,207,inf,upward,3939,0,2,inf,upward,2022-01,2022


## **3\. Carregamento**

- Com os dados manipulados, vamos fazer o seu download e carrega-los no Google Looker.

In [ ]:
df_cases.to_csv('./covid-cases.csv', sep=',', index=False)

### **ETL Vacinação covid**

**1\. Extração**

In [ ]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',')

In [ ]:
vaccines.head(2)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


- Assim como na primeira base, vamos começar a manipulação dos dados selecionando as colunas de interesse e as linhas referentes ao Brasil.

Nesta analise decidimos selecionar apenas os **dados referentes ao Brasil** e, além disso, **vamos selecionar apenas as colunas pertinentes**.

Agora sim vamos a extração. Ela deve demorar uns minutos pelo tamanho da base.

In [ ]:
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)

In [ ]:
vaccines.head(2)

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-05
1,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-06


In [ ]:
vaccines.info()
print(vaccines.isna().sum())
print((vaccines == '').sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1576 entries, 0 to 1575
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   location                 1576 non-null   object 
 1   population               1576 non-null   float64
 2   total_vaccinations       695 non-null    float64
 3   people_vaccinated        691 non-null    float64
 4   people_fully_vaccinated  675 non-null    float64
 5   total_boosters           455 non-null    float64
 6   date                     1576 non-null   object 
dtypes: float64(5), object(2)
memory usage: 86.3+ KB
location                      0
population                    0
total_vaccinations          881
people_vaccinated           885
people_fully_vaccinated     901
total_boosters             1121
date                          0
dtype: int64
location                   0
population                 0
total_vaccinations         0
people_vaccinated          0
pe

- Agora que os dados foram coletados é importante avaliarmos o conteúdo deste dataframe como formatos, se existem informações nulas, etc.

In [ ]:
vaccines.head(2)

,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-05
1,Brazil,215313504.0,NaN,NaN,NaN,NaN,2020-01-06


### **2\. Wrangling**

- Agora vamos começar tratando os dados faltantes, a estratégia será a de preencher os gaps com o valor anterior válido mais próximo.

In [ ]:
vaccines.fillna(0, inplace=True)

- Vamos também filtrar a nossa base de dados de acordo com a coluna `date` para garantir que ambas as bases de dados contém o mesmo período de tempo para análise.

In [ ]:
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

- Na sequência vamos alterar o nome das colunas.

In [ ]:
vaccines = vaccines.rename(
  columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  }
)

- Por fim, vamos criar agora novas colunas para enriquecer a nossa base de dados.

 - Chaves temporais:

In [ ]:
vaccines['year-month'] = vaccines['date'].apply(lambda date: date[0:7])
vaccines['year']  = vaccines['date'].apply(lambda date: date[0:4])

- Dados relativos:

In [ ]:
vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

- Garantir que o tipo do dado esta adequado ao contexto é fundamental para consistência da base de dados. Portanto, vamos fazer o *type casting* das colunas.

In [ ]:
vaccines['population'] =vaccines['population'].astype('Int64')
vaccines['total'] = vaccines['total'].astype('Int64')
vaccines['one_shot'] = vaccines['one_shot'].astype('Int64')
vaccines['two_shots'] = vaccines['two_shots'].astype('Int64')
vaccines['three_shots'] = vaccines['three_shots'].astype('Int64')

- Por fim, vamos reorganizar nossas colunas e conferir o resultado final.

In [ ]:
vaccines = vaccines[['date'
                           ,'country'
                           ,'population'
                           ,'total'
                           ,'one_shot'
                           ,'one_shot_perc'
                           ,'two_shots'
                           ,'two_shots_perc'
                           ,'three_shots'
                           ,'three_shots_perc'
                           ,'year-month'
                           ,'year']]

In [ ]:
# Função para formatar números com separadores de milhar
def format_number(number):
    try:
        # Converte para float e formata com separadores de milhar
        return f'{float(number):,}'
    except ValueError:
        # Retorna o valor original se não for um número
        return number

# Aplicar a formatação às colunas relevantes
vaccines['population'] = vaccines['population'].apply(format_number)
vaccines['total'] = vaccines['total'].apply(format_number)
vaccines['one_shot'] = vaccines['one_shot'].apply(format_number)
vaccines['two_shots'] = vaccines['two_shots'].apply(format_number)
vaccines['three_shots'] = vaccines['three_shots'].apply(format_number)

# Exibir as últimas 5 linhas do DataFrame formatado
vaccines.tail(5)



,date,country,population,total,one_shot,one_shot_perc,two_shots,two_shots_perc,three_shots,three_shots_perc,year-month,year
360,2021-12-27,Brazil,"215,313,504","329,011,365","165,952,037",0.7707,"142,764,283",0.6631,"25,218,893",0.1171,2021-12,2021
361,2021-12-28,Brazil,"215,313,504","329,861,730","166,062,249",0.7713,"142,965,728",0.6640,"25,758,909",0.1196,2021-12,2021
362,2021-12-29,Brazil,"215,313,504","330,718,457","166,143,380",0.7716,"143,282,084",0.6655,"26,219,623",0.1218,2021-12,2021
363,2021-12-30,Brazil,"215,313,504","331,164,041","166,185,628",0.7718,"143,398,692",0.6660,"26,507,937",0.1231,2021-12,2021
364,2021-12-31,Brazil,"215,313,504","331,273,910","166,195,505",0.7719,"143,436,012",0.6662,"26,571,077",0.1234,2021-12,2021


### **3\. Carregamento**

  - Com os dados manipulados, vamos fazer o seu download e carrega-lo no Google Looker.

In [ ]:
vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)

### **Visualização Interativa de Dados**

- Agora faz-se  necessário definir alguns *KPIs* (*key performance indicator*) para leitura dos dados. Definiu-se o seguinte:

**1.** Volume de casos e mortes;

**2.** Média móvel (7 dias) de casos e mortes;

**3.** Proporção de vacinados com 1ª, 2ª e 3ª doses.

- Além disso, faz-se necessário também incluir alguns gráficos para a análise exploratória de dados. Eles serão nomeados:

**1.** Distribuição do números de casos e mortes ao longo do tempo;

**2.** Distribuição da média móvel (7 dias) do números de casos e mortes ao longo do tempo;

**3.** Distribuição geográfica dos casos por estado por dia.

**4.** Distribuição da vacinação total por dia.

* O dashboard pode ser acessado em [EBAC - Projeto final COVID-19](https://lookerstudio.google.com/reporting/c649f065-d091-4ef9-ba45-589d9360eb86)

## **Storytelling**

* O ano de 2021, segundo da pandemia de Covid-19, foi marcado por pelo crescente número de casos pelo país. Dados do [Instituto butantan](https://butantan.gov.br/noticias/retrospectiva-2021-segundo-ano-da-pandemia-e-marcado-pelo-avanco-da-vacinacao-contra-covid-19-no-brasil#:~:text=O%20total%20de%20v%C3%ADtimas%20durante,dados%20fornecidos%20pelas%20secretarias%20estaduais.) detalham a segunda onda causada pelas flexibilizações e pelo surgimento de novas variantes mais transmissíveis. Entretanto foi um ano de esperança devido ao avanço da vacinação contra a Covid-19.

* Analisando o dashboard é possivel identificar a distribuição e tendência de casos e mortes acontecendo ao longo do ano. Embora o percentual de mortes seja inferior ao casos, constatamos média móvel de pelo menos 44 mortes a cada sete dias o quie foi suficiente para causar um estrago no sistema de saúde. Observando a distribuição por estados podemos notar que o sudeste foi extremamente assolado pelo vírus.